In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Intro to thought signatures

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fthinking%2Fintro_thought_signatures.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/thinking/intro_thought_signatures.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures.ipynb">
      <img width="32px" src="https://raw.githubusercontent.com/primer/octicons/refs/heads/main/icons/mark-github-24.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| Authors |
| --- |
| [Eric Dong](https://github.com/gericdong) |

## Overview

A thought signature is an encrypted representation of the model's internal reasoning process for a given turn in a conversation. By passing this signature back to the model in subsequent requests, you provide it with the context of its previous thoughts, allowing it to build upon its reasoning and maintain a coherent line of inquiry.

Thought signatures provide a powerful mechanism to maintain context in multi-turn conversations, enabling the model to tackle more complex, multi-step tasks that require reasoning and the use of external tools through function calling.


### Objectives

This tutorial explores the thought signatures feature of the Gemini API.


## Getting Started

### Install the Google Gen AI SDK for Python


In [1]:
%pip install --upgrade --quiet google-genai

### Authenticate your notebook environment

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Authenticate to Vertex AI on Google Cloud

You'll need to set up authentication by choosing **one** of the following methods:

1.  **Use a Google Cloud Project:** (Recommended for most users)
    - See instructions [Set up a project and development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)
2.  **Use a Vertex AI API Key (Express Mode):** For quick experimentation.
    - [Get an API Key](https://cloud.google.com/vertex-ai/generative-ai/docs/start/express-mode/overview)
    - See tutorial [Getting started with Gemini using Vertex AI in Express Mode](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_express.ipynb).

This tutorial uses a Google Cloud Project for authentication.

In [3]:
import os

# fmt: off
PROJECT_ID = "veo-testing"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
# fmt: on
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "global"

### Import libraries


In [4]:
from IPython.display import Markdown, display
from google import genai
from google.genai.types import (
    Content,
    GenerateContentConfig,
    Part,
    ThinkingConfig,
    Tool,
)

### Connect to the generative AI service on Vertex AI

In [5]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Supported Models

Learn more about [supported Gemini models](https://cloud.google.com/vertex-ai/generative-ai/docs/models). This tutorial uses the Gemini 2.5 Flash model.

In [6]:
MODEL_ID = "gemini-3-pro-preview"  # @param {type: "string"}

## Use Thought Signatures

When thinking is enabled, the API response includes a `thought_signature` field containing an encrypted representation of the model's reasoning. When a function's execution result is sent back to the server, including the `thought_signature` allows the model to restore its previous thinking context, which will likely improve function calling performance.

Example:

```
thought_signature=b"\n\xa9\x06\x01\xcb...'
```


### Use Cases
The primary and most powerful use case for thought signatures is to enable sophisticated, multi-turn interactions with external tools. Imagine a scenario where a user's request requires multiple steps to resolve, such as:

- Finding the current weather.
- Based on the weather, deciding on an appropriate activity.
- Booking a reservation for that activity.

Each of these steps might involve calling a different function. Without thought signatures, the model would treat each step as an isolated request, losing the crucial context of the user's initial intent and the results of previous function calls. With thought signatures, the model can chain these calls together, understanding the dependencies and making informed decisions at each step.


## Example: Conditional Thermostat Control

In this scenario, a user wants to set a thermostat based on the current weather. The request is: "If it's too hot or too cold in London, set the thermostat to a comfortable temperature."

This requires the model to:

- Call a tool to get the weather in London.
- Use the returned weather information to decide if another tool needs to be called.
-Call the tool to set the thermostat if the condition is met.

### **Step 1**: Define Functions and Tools

First, we define the two functions the model can use:  `get_current_temperature` and `set_thermostat_temperature`.

In [7]:
# Mock Function Implementations


def get_current_temperature(location: str) -> dict:
    """Gets the current weather temperature for a given location."""
    print(f"Tool Call: get_current_temperature(location={location})")
    # Mocking a real API call.
    return {"temperature": 30, "unit": "celsius"}


def set_thermostat_temperature(temperature: int) -> dict:
    """Sets the thermostat to a desired temperature."""
    print(f"Tool Call: set_thermostat_temperature(temperature={temperature})")
    # In a real app, this would interact with a thermostat API.
    return {"status": "success"}


# Function Declarations for the model


get_weather_declaration = {
    "name": "get_current_temperature",
    "description": "Gets the current weather temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {"location": {"type": "string"}},
        "required": ["location"],
    },
}

set_thermostat_declaration = {
    "name": "set_thermostat_temperature",
    "description": "Sets the thermostat to a desired temperature.",
    "parameters": {
        "type": "object",
        "properties": {"temperature": {"type": "integer"}},
        "required": ["temperature"],
    },
}

# Create a single Tool with both functions
# The model will choose which, if any, to call.
thermostat_tools = Tool(
    function_declarations=[
        get_weather_declaration,
        set_thermostat_declaration,
    ]
)

### **Step 2**: First Turn - Get the Weather

We send the initial prompt to the model. We expect it to call the `get_current_temperature` function and return a thought signature to maintain the context of the user's conditional request.

In [8]:
# The user's initial multistep request
prompt = """
If it's too hot or too cold in London, set the temperature to a comfortable level.
Make your own reasonable assumption for what 'comfortable' means and do not ask for clarification.
"""

contents = [Content(role="user", parts=[Part(text=prompt)])]

# Send the first request
config = GenerateContentConfig(
    tools=[thermostat_tools],
    thinking_config=ThinkingConfig(
        include_thoughts=True,  # Optional, include_thoughts can only be enabled when thinking is enabled
    ),
)

response_turn_1 = client.models.generate_content(
    model=MODEL_ID, config=config, contents=contents
)

In [9]:
response_turn_1

GenerateContentResponse(
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            text="""**Defining the Approach**

I've firmed up the strategy. I'll begin by using the `get_current_temperature` tool with the location set to London. Subsequently, I'll assess the temperature returned. If the temperature indicates it's uncomfortable, I will adjust the thermostat with `set_thermostat_temperature` to 21 degrees Celsius. I'm ready to move forward.


""",
            thought=True
          ),
          Part(
            function_call=FunctionCall(
              args=<... Max depth ...>,
              name=<... Max depth ...>
            ),
            thought_signature=b'\n\xeb\x10\x01\xe3\xf1\xff^\x9b\x1e\xc5\xe4\x978\x82"\x13\x00-\xa0\xe1\xe7\x02S\x95\x18\xc6lE_v\xb3\xdc\xdef\xbd]\xbev\xbfw[\x9e\xa8\x86\r&\xdbo\xf0\xca\xae\xef\xb9X1B*\xae\x95\x0e\'\xe5\x83\xe8\xb43\xa5\x14\x0e#\x99\x96B\xe3_I2\xb9\xc7\x11+\xdf\xbeP\xb5\x01\xbd\x891\x8a\xf4n\x93\xa0>

In [10]:
def process_tool_call(response):
    """Processes a model's response to extract and execute a tool call.

    Args:
        response: The response object from the model.

    Returns:
        A tuple containing the tool_call object and the result of its execution.
        Returns (None, None) if no function call is found.
    """
    tool_call = None
    result = None

    # Extract the function call, thought signatures, thought summary
    # from the response parts
    for part in response.candidates[0].content.parts:
        if part.function_call:
            tool_call = part.function_call
        if hasattr(part, "thought_signature") and part.thought_signature:
            display(Markdown(f"**Thought Signature**: {part.thought_signature}"))
        if part.thought:
            display(Markdown(f"**Thought Summary**: {part.text}"))
        else:
            display(Markdown(f"**Answer**: {part.text}"))

    if tool_call:
        if tool_call.name == "get_current_temperature":
            result = get_current_temperature(**tool_call.args)
            display(Markdown(f"**Tool Response**: {result}"))
        elif tool_call.name == "set_thermostat_temperature":
            result = set_thermostat_temperature(**tool_call.args)
            display(Markdown(f"**Tool Response**: {result}"))
    else:
        print("No valid tool call found in the response.")

    return tool_call, result

In [11]:
# Process the response
print("--- Turn 1 ---")
tool_call_1, result_1 = process_tool_call(response_turn_1)

--- Turn 1 ---


**Thought Summary**: **Defining the Approach**

I've firmed up the strategy. I'll begin by using the `get_current_temperature` tool with the location set to London. Subsequently, I'll assess the temperature returned. If the temperature indicates it's uncomfortable, I will adjust the thermostat with `set_thermostat_temperature` to 21 degrees Celsius. I'm ready to move forward.




**Thought Signature**: b'\n\xeb\x10\x01\xe3\xf1\xff^\x9b\x1e\xc5\xe4\x978\x82"\x13\x00-\xa0\xe1\xe7\x02S\x95\x18\xc6lE_v\xb3\xdc\xdef\xbd]\xbev\xbfw[\x9e\xa8\x86\r&\xdbo\xf0\xca\xae\xef\xb9X1B*\xae\x95\x0e\'\xe5\x83\xe8\xb43\xa5\x14\x0e#\x99\x96B\xe3_I2\xb9\xc7\x11+\xdf\xbeP\xb5\x01\xbd\x891\x8a\xf4n\x93\xa0>Z%\xe7\xe0\x9d\xef\x06\xa5\xa8X\x08\x1cf\n\x16\xd8\xe5\x81\xf6\xb4\x9av\x00-\xe05S\xa4\xa7S1`\xb3\xe6\xfc\x95\x92\x19\xe6\xc14\x9a\x7f|[A\xc1~y\x18\xc4\xd7\xb9>\x15q\xe0\xa9\xb9\xa2\xb9\x1f%\x14\x8f)9(\x91[\xf3\xdc\x0c+\xb0fj\x1cI\x0e\xbf\x1b\xd8\xc4\xd6\x07\xd8\x9fl\xe3\x19\n\xfa\xa6\xa4\xa3bw&\x02N\xf7Kf\r\xc3\xca|\xb2\xbcq`\xdd:\x05|`\xf2\xfa\xdd\xd98x\xb0\xc8\xaa\xaf\xad\xf0v"\x1eh$\xcf\xac\xda\xd0\x89\xcd>T~\xa9\xa8\xa6\xc4f\xaaz\xcc\xad\xf9\xf4\x05\xc0dP\x9fB\xce\x91$\xdd\xef\xb6\xa5\xee\xef\xc5\xb5\x83o2["\xd5\xa0\x9a\xfa\x11\xb1\xa7\xedD3\xc9\x18K\x9f0l3\xd9\xd7A|>{F5\x9f\xbd\x88_e\x13\xca<\xdd\xb2\x10\xa9\x05\x99\xbfw#\x95\xf7\x9c\x00W\xb6$\x04\xcf\x86\xedE\x92\x90\xa5\xdf\x87Z\x94\xc5\xb5;\xf2)\x87f\xb3\xaa\xd3.\xbd\xc4Q\xd2o\xa6\xed|\x1c<\x1f\xb38\xd4\x04\x97\xa9p2J\xadyk\x06\xbd\xffLP\xac]j\xc564\x85t\xe8+\xdc<\x9c\xb4\x10#\xc8v\xeb\xa0`\xa6L\x97\xd0\xf8r9kMp\xda\x12\xa0k\xb1\x17\xea\x0b0Hy\xfa\x15\\\xee\xa2D\xf7\x9cc\xf0QB\xe5\xa9Z\xe1\xe6{\xed\xa5\x1e\xa2lCv\xad\xe4\x95w\xd1\x1d\x04z\xbe\xfbm\x98\x97*\x88\x85\x18\x1f|\xd1\x8e\x93\x15\x1a\x8bo\xb9\xcfJ\t\xe2\x01\xa5i\x91`!{t\x93H\xe7\xe9\x16s\x95&\x12\xc7\x99\x91\xe3HX\xad*)\xee$&TWS\xbc\x156\xe3\xedd\xe7\xc7_\xbc\xb1|\'\xd3\xbe>\x9f\xdd\xa0\xce\xbd/p\xa6$;|\xbcw\x81\x9b\xa62\xf8\x9d\x9d@,`\xbb\xd3\xcb\xbe\xc9y\xf0\x16\x98m~\x99\x87\xea\xc3\xd0|\xe4O\xe7\xb3\x0c(\x01\xd1\x8fn\xa5s\xce\xc5\x83\x1be\xb3}\x15@\xb6\xfbp\xbdCw\xf9\xda\xa0\xe1\x9c\xa1\xaa\x86\xb1\x9e\xf1\xd40\x9c\xa2@\xd2\x80\x8c\xa9\x07\xfe\x07x\x99\x9b\x90\xd9\x80\xc1\xf1\xdb\xdc\x8a\xec5\xb2\xf5e4\x82qu_\x94\xfa`V6\x11~\x86]\xdb\x15\xc8\xe8\xf0\xd3\x98fD;`\x83+\x1bKg\xa5\x10I\r\xbe\x82\x07\xa8\xc8\xdbl\xdd:\t\xc5\x8e\xd6\xc500\x07S\xcae\xc0Z\x98\x07\x87>A\xec\x06\xe5;`\xed\x11\x9c\x11\xbd\x14\xbc\xdc\xeb\xa8T\x02A\xaf\xbc\xd6\xf5\x0c5\xe1(\xcb\xe8\xb7\xe1%\x1dB\xab\xf0\x0c\xdd\xf5\xeb\xbd\xeeD\xa9\x8c\xcf\x04\xa1x\xd1\xbc9\xe90\xd3`\xe9Q\xbe\xbf\xf8\xacK3\xc2\xe0(\xc8\x8c\xfc\x1d\xa9\xb1\x8a0\xb3\xed\xbfz\xcap\xaci\xab:\r\t\x8c\xbeJ\x00B\x16\xcf`\x0b\x8b\xb2P\xb3\x90N\xf0H\x8ceh\x05\xf7\xd0\x05Gv\xa0Fe\xfc\x10\xc4\xe1\xda\xc4\x88}\xd0#t\x8e\xd7\x84p\x10\xcf=3\x93}\xb6%9e\xf3\xa1j7}\x02\xfdXL\xa6\x03\xd5\x1c\x86\xec\xb0\xcd\xb1\x0fC\xbem\x15r\xb5\xd4\x11{r\xbbl\x13\xd1\xbc\xd0\xf1\x10\xc0\x9bw\x92Y\x07~\x92\x12\xba\xf3\xcb`!zt\x1f\xc6Z%\x90!\x90\xa6\xe4*\xefh\xbc?\x92\x0e\xbb\xb6Rj)\xf4\xff2\x9ewU#Xn;\xbbL\x00\x1a%\t\xf8\xf7\xf7-\x9c\xf2n\x1b\xf6b\xfb5T\x86wjwb|\r\x0e*\xaa&b$\xa5\xab\x8b!\x88,\xf4la\x00\x17=\xd0hFWB\xb7\x9dP\x84\xd5&\xda\x8dK\xe7Y\x1e\xcc\xbey\xd9\x11Y\x85\x83\xe2\x95\xa8\x90\x9d\xd0(\xeaX\x96ODTwD"\xc2?\xd3\x80\x15\x13t,"G\'Y\x12\xdb\xecM?=\x93`\x88\xc6\x04\xee\xee\x07\xa0\x95|h/\xa5q_\xcfi\xba\x1a\xe4{QG\xb7C\xe9f\xf9\x1c\xb1Z\xec\xa1c\x9f\xf0\xc7h"\xf9\xbe\xdfa\xee\x97\x1d\xc2\x06]\xc1O\x96\xe9\xf5g=\xe5\xbd\x01\xa3\x07\x93\xce\xf5Ft\xc5F\x9b\xa25\x9f=\xbfe\x80\x8c\x941\xcb\x90\x7fIb7\xd1\xdaU\x02h\x11\x9e\xfa\xacWj\xfa\xb8\x13\xe8\x80@\x849;s3\xc5\xa3\xbb\x04T\x08\xe4\xb5\xd7\xa7\xce%\x80P\x17\xe7p\x1d4"\xd1\x88\x838\x9fQ\rQ\xef\x16\xc5\xd4\xd8m5\x17_\x15\xd6\xf4\xabFg\x97 \x90\xf1j\xfeK\x7fC\xde #\xdbS;\xb4\xbd\xe6\x91\xa7\xb9\xd1\x98\x13\x98\x940\x0e&\xbf\xceo5\xab\x9b\xb0\\,s}T\xa4\x1e\xb0XJ\xe3\xb3\x92\x992\\Ea\x8e\x0ceA\xcdk\xf2\xc2W\xeftns(\x12\xbcy\xf0Ze\xda)\x87d=\xf1\xbd\x80\xe9o\x02\xa0QnL\xf8\x85b\xf1\x9c\'w\x07S$\x070cjI\x05\xe7\xb2M\xa1\xb8\x19\x83\xc8P\x10+]-\xf4\xd7\x9a\x8dY3\t}@Q\x92\xae\x87\xb3\x17\xd9\xb9:\xb6E\xd5\xc8@U\xd9\xa4y\xa9\x90\xdc\x7f\xc1\xd0\xb2\xca\xb9\xa8\x08\xb5\x9b\xdb\xad\xad+~\xa4\xc1\x8c\xa3\x03\xd5\x1a3g\xbb@\xa9\x17.\x8a\xf1\\\xaf\x12\xfd\xa7\x15\xdf\xba\x977\x00j\xc4a\xd3%\x07\x83\xc0(B\xa9Z\\\xb5\x1b\xa6LJ;\xb5\xdf/\xfd\x1a6u\x0f>5\xf3\x85Z\xe9\x93\xe0Z\xab\xe4\x1c\x06\x9a\x97\x91 :#\xa8\x04r\x06\x02\x99\x88:T\x0b\t\xb9\xbdm\xad\xc5\xf8\xadK8\xc8\xf0\t\x8b"\x1e\xe4[\xdc\xaf\x91~\xe6\xb8b\xc8p\xac\x0b\t@\tu>\xaf\xa6\xa9\n\\f6g\xf5\x9c\xea\x8ah\x82\xcc\x1b3\xd3\xcf5\x9f\x8c\x8f\xf4\xe4\xa5OR\x9cB\x8c\xdb\xdf\xbc\x12\xe0\xfbf\x1b\x0fT8\x8c\xba$\xf2\x97\xaa\xaf\xbeG\x8dy\xd3ZrK\xa8\x04\x0b\x19\xe6h\xe7\xa5\xa0\x98\x0c\x0b\xdc\x1d\x9b6\xaa\x96\xe3U9\xbbA\x02\xea\xf4|\xccy\x17\xd9\'f\x05\xba8\xb6i\xda\x1a\\e\x19>=\x87\xcf\x83\xf1R\x19\x91\xea\xad\xf7C\x02\xac\xfb\xa6\xc5{6\xd0\xa7)\xc6\xb5\x89$\xcaGLz|e>\x01\xa1/\x91\x88\xcf\x84V\xea\xd7\xa8\xb2>\xf0\xa1\xc2{Qi\xe6\xb9\xdb\x8f\xa5\x7fYP\xf7\xb0\xcb0I\x9a\xb2A\x10^\xa0!\x97N\x9e\x86c\xf8\xa8e\xa2\x85\xf4o\x96\x84\x07\x11\xfb\x1f\xea\x8e\x14%\x1d\x8b\x04\x08]\xa6\xebtJ9\x9f;\x18\xa7\x05}\x9f\x83\xfd\xc4\xeb\xf6jD\xf0\xe82\xf6\x01\xaa\xb6\x8cgG^]\xcf-\x8e\xbd\xb4\xdcn\xd5\x8d\xc2\x10\xfb\xdb\xaf\\Gq\x9dBR\xf0<{g\xac\xaa8\xf9c\xd1\x9b"\x17\xfb\xb0b\x18\x1d\rZ.?\n\xf3\xfbV\xd3\xd8!\xb15\x85\x15\xdfj\xbeK%\xf5\xe8\x1a\x99\xdfQcJ"UGf\xba@\x80\xda\xcd\xb6\xb31>\x0e\xd9\xe1\r\xd8\xfaN\x0baj\x83\xf9{\x88\xdd\xf3\x1a\xc4\xd8\xc6\xccI\xbe\x97q\xb7\x95\x84|\x9d\xf0\x15\xaeJ"J\xa91~\xe5*D\x19J\x9a)\xfdG\xe0\xee*\x9d\xdb\xebh?\x0eN&\x17\x0b\xd1\xd2\xf1s\x85\xfc\r\x053}\x15\x00\xa5\x95\xa3\x87\xeaQ#\\\r\xefW\x8f\xaal\xf4\xb8\x10\xaf\x1e/\xf0\x03@[\x05\xde78\xd0&q8\xcb\xd0O\xb5\x13\x95\t\x15\x13\xe8\xbai\x11\x08.\xef#&\x81\x18?Vj\x86\xa2\xd5\xa4S\x0cZ\xb2\xdd3\x14\xf6\'OE2GL\x97\xef&\xd3\xf0\x83\x9b\x96\xa8\xac\n\x19\x05\xd2\t\xe1b\xc9EQ\xac\xdd\xb8W\x0c3\xbf)m;\xe6B\xbb\n\xed\xd4\x01\xac\x94\x06Xs\xf0\xbfN\x1e0\xd6\x8d\xa1\x0b\xcd\xc6\xcf\xafOi\x80?\x9dc@\x00\xd05\x87\xd6`\x0f\xfa\xe1\x81\xf4\xf4\x0b: \xc3\x0f,JO+\xc8\xb51\xc5\xbb\xec\x8eF%P\x93\x0c!\x80\x01h\x8a+\xcc6\xe3\x12(\x0c\xaa\xef\xc8\xe7\xeb\x1dn\xe0w\xcfIq\x1bU\xbe?\x90\\.\xcf#S\xfc\x93\x92]\x11\xa9\xebb\xd8\xcf\xdc\xa7c\x98\r\x86P+\xb7\x18_\x17\x99JZ\x9e\x98\x0b\xc6\xdb\xa2\xcb\xa6[\xf6j\x8b\x96\xa1\xf9g\xe1\xae#\x88}\x86u\xc3);\xa5=.\x83\x1e{\x8e\xae\t\xcb\x17j4\xd7\x81\xe6\xe8z\xd1\xfb\xf7\x1fR\xdc\xcfJ*\xaa \xfdj\x06\x81!\x9b\x7foE~^m\xbbPu\x8c\xd2d\x93 \x97tS\x9c\x19\x98\r\xc1[\xcba\x8e3\xfd\x11\xc5\xb3\xd0\x95\xcbp\xaa\xc4\xf6\xcfa\xba\xff7\x9d'

**Answer**: None

Tool Call: get_current_temperature(location=London)


**Tool Response**: {'temperature': 30, 'unit': 'celsius'}

### **Step 3**: Second Turn - Set the Thermostat

Now, we send the model's previous response (containing the thought signature) and the result from our first tool call back to the model. Using this context, the model will decide if it needs to call the `set_thermostat_temperature` function.

There is no additional work for you to do. The `"thought_signature"` part will be sent back in the model turn's `Content`, it is also handled transparently by
automatic function calling.

In [14]:
tool_call_1

FunctionCall(
  args={
    'location': 'London'
  },
  name='get_current_temperature'
)

In [15]:
result_1

{'temperature': 30, 'unit': 'celsius'}

In [12]:
contents_bk = contents.copy()
print(contents_bk)

[Content(
  parts=[
    Part(
      text="""
If it's too hot or too cold in London, set the temperature to a comfortable level.
Make your own reasonable assumption for what 'comfortable' means and do not ask for clarification.
"""
    ),
  ],
  role='user'
)]


In [13]:
contents_bk.append(response_turn_1.candidates[0].content)
print(contents_bk)

[Content(
  parts=[
    Part(
      text="""
If it's too hot or too cold in London, set the temperature to a comfortable level.
Make your own reasonable assumption for what 'comfortable' means and do not ask for clarification.
"""
    ),
  ],
  role='user'
), Content(
  parts=[
    Part(
      text="""**Defining the Approach**

I've firmed up the strategy. I'll begin by using the `get_current_temperature` tool with the location set to London. Subsequently, I'll assess the temperature returned. If the temperature indicates it's uncomfortable, I will adjust the thermostat with `set_thermostat_temperature` to 21 degrees Celsius. I'm ready to move forward.


""",
      thought=True
    ),
    Part(
      function_call=FunctionCall(
        args={
          'location': 'London'
        },
        name='get_current_temperature'
      ),
      thought_signature=b'\n\xeb\x10\x01\xe3\xf1\xff^\x9b\x1e\xc5\xe4\x978\x82"\x13\x00-\xa0\xe1\xe7\x02S\x95\x18\xc6lE_v\xb3\xdc\xdef\xbd]\xbev\xbfw[\x9e\xa

In [14]:
contents_bk.append(
    Content(
        role="tool",
        parts=[
            Part.from_function_response(
                name=tool_call_1.name,
                response=result_1,
            )
        ],
    )
)
print(contents_bk)

[Content(
  parts=[
    Part(
      text="""
If it's too hot or too cold in London, set the temperature to a comfortable level.
Make your own reasonable assumption for what 'comfortable' means and do not ask for clarification.
"""
    ),
  ],
  role='user'
), Content(
  parts=[
    Part(
      text="""**Defining the Approach**

I've firmed up the strategy. I'll begin by using the `get_current_temperature` tool with the location set to London. Subsequently, I'll assess the temperature returned. If the temperature indicates it's uncomfortable, I will adjust the thermostat with `set_thermostat_temperature` to 21 degrees Celsius. I'm ready to move forward.


""",
      thought=True
    ),
    Part(
      function_call=FunctionCall(
        args={
          'location': 'London'
        },
        name='get_current_temperature'
      ),
      thought_signature=b'\n\xeb\x10\x01\xe3\xf1\xff^\x9b\x1e\xc5\xe4\x978\x82"\x13\x00-\xa0\xe1\xe7\x02S\x95\x18\xc6lE_v\xb3\xdc\xdef\xbd]\xbev\xbfw[\x9e\xa

In [15]:
# Append the model's function call message from turn 1 (this includes the signature)
contents.append(response_turn_1.candidates[0].content)

# Append the result of the function execution from turn 1
contents.append(
    Content(
        role="tool",
        parts=[
            Part.from_function_response(
                name=tool_call_1.name,
                response=result_1,
            )
        ],
    )
)

# Send the second request
response_turn_2 = client.models.generate_content(
    model=MODEL_ID, config=config, contents=contents
)



In [16]:
response_turn_2

GenerateContentResponse(
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            text="""**Setting the Thermostat**

I've determined that the current temperature in London is excessively high, exceeding what I consider comfortable. Based on this, I've decided the desired temperature setting should be 21°C. Now, the next step is to implement that change.


""",
            thought=True
          ),
          Part(
            function_call=FunctionCall(
              args=<... Max depth ...>,
              name=<... Max depth ...>
            ),
            thought_signature=b'\n\xab\x02\x01\xe3\xf1\xff^\xcb\x14E\xabFS\xef\x13\xa7\x85\x80"x\xe36\x08\x91\xbei_&]@Z\xf9\x84\x1b\xfa\xa8s\xc3[\x88\xad\xe9\xec$[[\xe5v\x81j2\xb7\x9b\xb7\xce\x94\xd5\x80\xed\x91\xc2=\xe4\x88\x17&\xbd\xe7<\xbe<_\xba\x90\xfd\xbd\x17\xf7\x11\x01\x8a\xab\x96v\x0b&\x07\x17\xdc=&\xdc;\x1f\xb9T...'
          ),
        ],
        role='model'
      ),
      finish_reason=<Finish

In [17]:
# Process the response
tool_call_2, result_2 = process_tool_call(response_turn_2)

**Thought Summary**: **Setting the Thermostat**

I've determined that the current temperature in London is excessively high, exceeding what I consider comfortable. Based on this, I've decided the desired temperature setting should be 21°C. Now, the next step is to implement that change.




**Thought Signature**: b'\n\xab\x02\x01\xe3\xf1\xff^\xcb\x14E\xabFS\xef\x13\xa7\x85\x80"x\xe36\x08\x91\xbei_&]@Z\xf9\x84\x1b\xfa\xa8s\xc3[\x88\xad\xe9\xec$[[\xe5v\x81j2\xb7\x9b\xb7\xce\x94\xd5\x80\xed\x91\xc2=\xe4\x88\x17&\xbd\xe7<\xbe<_\xba\x90\xfd\xbd\x17\xf7\x11\x01\x8a\xab\x96v\x0b&\x07\x17\xdc=&\xdc;\x1f\xb9T\x1e\xfd\x89\xb1Z\x14\xfa\x0b4f\xc9=\xce\xcdG\x80 \xaam\xcc\xe0\x14\xec\x13\xca\xcft\x11\xf6\xc7\x87\xb5\x822\xbfu\x0c\x9a\xbaL1\t\xebG|Z\x1b\xf1$G[\xe9\x12\xbd\x06\xc1\xb5\xf3\xe4-I\x16\x0e\x0cxy\xa4\r\x96\x89\x12\xfc\xe5\xfc\x81\x8c\xdc\xe7a`S\xe0\x9a\xda\x1e\xea\xa2-\x9f\xbc\xd2\xe1G\xdey0[\x9f\xd4y\xdb\xa9\xb7\x0e\xf8\x0b\xc9\xff<t\xc9-\xab\x96\xb9\x10\xb1f]\xc9UP\x16\xec\x12V\xdf-\x0e\x84n\xcf\xfe\x02|\x11S\xf8\xda\xb5\xe0\xc7\x0b\xa2\x91\x91\xae4:M\x82\x1e\x95Fn\xe5\x07\x83Y4\xef\x17\xde\xae\xb9p\xc1\xb37\xaa\xa7|i\xe6+_\x1d?\'Q\t\x82\x14^\x06\xa1\x95\xa13pZ\xccR\x14"3\xf6\x86k`\x83\xbc\xb0\x9c"'

**Answer**: None

Tool Call: set_thermostat_temperature(temperature=21)


**Tool Response**: {'status': 'success'}

In [18]:
tool_call_2

FunctionCall(
  args={
    'temperature': 21
  },
  name='set_thermostat_temperature'
)

In [19]:
result_2

{'status': 'success'}

### **Step 4**: Generate a user-friendly response

Generate a user-friendly response, informed by the action it just took.

In [21]:
contents_bk = contents.copy()
print(contents_bk)

[Content(
  parts=[
    Part(
      text="""
If it's too hot or too cold in London, set the temperature to a comfortable level.
Make your own reasonable assumption for what 'comfortable' means and do not ask for clarification.
"""
    ),
  ],
  role='user'
), Content(
  parts=[
    Part(
      text="""**Defining the Approach**

I've firmed up the strategy. I'll begin by using the `get_current_temperature` tool with the location set to London. Subsequently, I'll assess the temperature returned. If the temperature indicates it's uncomfortable, I will adjust the thermostat with `set_thermostat_temperature` to 21 degrees Celsius. I'm ready to move forward.


""",
      thought=True
    ),
    Part(
      function_call=FunctionCall(
        args={
          'location': 'London'
        },
        name='get_current_temperature'
      ),
      thought_signature=b'\n\xeb\x10\x01\xe3\xf1\xff^\x9b\x1e\xc5\xe4\x978\x82"\x13\x00-\xa0\xe1\xe7\x02S\x95\x18\xc6lE_v\xb3\xdc\xdef\xbd]\xbev\xbfw[\x9e\xa

In [22]:
contents_bk.append(response_turn_2.candidates[0].content)
print(contents_bk)

[Content(
  parts=[
    Part(
      text="""
If it's too hot or too cold in London, set the temperature to a comfortable level.
Make your own reasonable assumption for what 'comfortable' means and do not ask for clarification.
"""
    ),
  ],
  role='user'
), Content(
  parts=[
    Part(
      text="""**Defining the Approach**

I've firmed up the strategy. I'll begin by using the `get_current_temperature` tool with the location set to London. Subsequently, I'll assess the temperature returned. If the temperature indicates it's uncomfortable, I will adjust the thermostat with `set_thermostat_temperature` to 21 degrees Celsius. I'm ready to move forward.


""",
      thought=True
    ),
    Part(
      function_call=FunctionCall(
        args={
          'location': 'London'
        },
        name='get_current_temperature'
      ),
      thought_signature=b'\n\xeb\x10\x01\xe3\xf1\xff^\x9b\x1e\xc5\xe4\x978\x82"\x13\x00-\xa0\xe1\xe7\x02S\x95\x18\xc6lE_v\xb3\xdc\xdef\xbd]\xbev\xbfw[\x9e\xa

In [23]:
contents_bk.append(
    Content(
        role="tool",
        parts=[
            Part.from_function_response(
                name=tool_call_2.name,
                response=result_2,
            )
        ],
    )
)
print(contents_bk)

[Content(
  parts=[
    Part(
      text="""
If it's too hot or too cold in London, set the temperature to a comfortable level.
Make your own reasonable assumption for what 'comfortable' means and do not ask for clarification.
"""
    ),
  ],
  role='user'
), Content(
  parts=[
    Part(
      text="""**Defining the Approach**

I've firmed up the strategy. I'll begin by using the `get_current_temperature` tool with the location set to London. Subsequently, I'll assess the temperature returned. If the temperature indicates it's uncomfortable, I will adjust the thermostat with `set_thermostat_temperature` to 21 degrees Celsius. I'm ready to move forward.


""",
      thought=True
    ),
    Part(
      function_call=FunctionCall(
        args={
          'location': 'London'
        },
        name='get_current_temperature'
      ),
      thought_signature=b'\n\xeb\x10\x01\xe3\xf1\xff^\x9b\x1e\xc5\xe4\x978\x82"\x13\x00-\xa0\xe1\xe7\x02S\x95\x18\xc6lE_v\xb3\xdc\xdef\xbd]\xbev\xbfw[\x9e\xa

In [24]:
# Append this turn's messages to history for a final response
contents.append(response_turn_2.candidates[0].content)
contents.append(
    Content(
        role="tool",
        parts=[
            Part.from_function_response(
                name=tool_call_2.name,
                response=result_2,
            )
        ],
    )
)

# Get the final text response
final_response = client.models.generate_content(
    model=MODEL_ID, config=config, contents=contents
)

display(Markdown(f"**Final Response**: {final_response.text}"))

**Final Response**: The current temperature in London is 30°C, which is quite hot. I have set the thermostat to a comfortable 21°C.

In [25]:
final_response

GenerateContentResponse(
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            text='The current temperature in London is 30°C, which is quite hot. I have set the thermostat to a comfortable 21°C.'
          ),
        ],
        role='model'
      ),
      finish_reason=<FinishReason.STOP: 'STOP'>
    ),
  ],
  create_time=datetime.datetime(2025, 11, 26, 13, 21, 6, 634096, tzinfo=TzInfo(0)),
  model_version='gemini-3-pro-preview',
  response_id='wv4mafDZJvO24_UP9PW30QQ',
  sdk_http_response=HttpResponse(
    headers=<dict len=10>
  ),
  usage_metadata=GenerateContentResponseUsageMetadata(
    candidates_token_count=31,
    candidates_tokens_details=[
      ModalityTokenCount(
        modality=<MediaModality.TEXT: 'TEXT'>,
        token_count=31
      ),
    ],
    prompt_token_count=255,
    prompt_tokens_details=[
      ModalityTokenCount(
        modality=<MediaModality.TEXT: 'TEXT'>,
        token_count=255
      ),
    ],
    total_token_

This multi-turn process, held together by passing the model's response history (including the encrypted thought signature) back each time, allows the model to perform complex, sequential tasks.